In [1]:
using JuMP, Plots, HiGHS, Statistics, LinearAlgebra, LaTeXStrings, DataFrames, CSV, XLSX, StatsPlots, NPZ, UnPack, JLD2, PyCall, Random
include("constants.jl")

0:100:10000

In [2]:
ver_ptr = HiGHS.Highs_version()       # returns a pointer
ver_str = unsafe_string(ver_ptr)      # convert pointer to actual string
println(ver_str)

1.8.0


In [3]:
include("opt_models.jl")
include("outputs_processing.jl")

plot_wind_power_scenarios_and_boxplots (generic function with 1 method)

##### Reserve market

In [4]:
time0 = time()
results_RI_NI = reserve_RI_NI(NI_up_og, NI_down_og, Cf_Rup_og, Cf_Rdown_og)
time1 = time()
println("Time for RI_NI optimization: ", time1 - time0, " seconds")
results_RI = reserve_RI(Cf_Rup_og, Cf_Rdown_og)
rf_up, rf_down = results_RI_NI[end].rf_up, results_RI_NI[end].rf_down
q_up, q_down = get_your_qs(results_RI_NI[end], results_RI[end])
Lambda_r_up, Lambda_r_down, fig_down = merit_order_price_reserve(rf_up, rf_down, Cf_Rup_og, Cf_Rdown_og; dir = "down regulation", fontsize= 21)
_,_, fig_up = merit_order_price_reserve(rf_up, rf_down, Cf_Rup_og, Cf_Rdown_og; dir = "up regulation", fontsize = 21)
Lambda_R = (Lambda_r_up, Lambda_r_down)
R_bids = [rf_up, rf_down]

nb of variables 16
nb of constraints 34
Binary variables: 0
Time for RI_NI optimization: 4.960999965667725 seconds


2-element Vector{Vector{Float64}}:
 [30.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
 [0.0, 0.0, 0.0, 0.0, 40.0, 40.0, 40.0, 40.0]

#### Scenario Generation

In [5]:
file_list = ["scenario_generation/seasonal_scenarios_0125.csv"]

Pw, centers, Prob, nS_all = preprocess_scenario_files(file_list; season="Spring")         # Considering that they all have the same uncertainty

hour = 1
prob = Vector{Float64}(undef, nS_all)
prob = round.(Prob[1][:, hour], digits = 5)     # in this case we prob is only constituted of 1 element

k = 10
fold_size = div(nS_all, k)
Pw_hour = Pw[hour]
n_hours = size(Pw)[1]

24

### Benchmark Model

In [6]:
time1 = time()
res_BM = run_kfold_simulation("Benchmark", Pw, nS_all, prob, 0, 0, k, fold_size, 
                            rf_up, rf_down,NI_up_og, NI_down_og, Cf_Rup_og,  Cf_Rdown_og, R_bids, Lambda_R, hour)
time2 = time()
println("Time for Benchmark k-fold simulation: ", round(time2 - time1, digits=2), " seconds.")

Fold: 1
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.46199989318847656 seconds
Fold: 2
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.01699995994567871 seconds
Fold: 3
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.010999917984008789 seconds
Fold: 4
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.026000022888183594 seconds
Fold: 5
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.017999887466430664 seconds
Fold: 6
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.01100015640258789 seconds
Fold: 7
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.018000125885009766 seconds
Fold: 8
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benchmark time: 0.046000003814697266 seconds
Fold: 9
nb of variables 543
nb of constraints 1133
Binary variables: 0
Benc

#### Base Case FCB Model

In [7]:
time3 = time()
res_GDCA = run_kfold_simulation("TA_bin_GDCA", Pw, nS_all, prob, q_up, q_down, k, fold_size, 
                            rf_up, rf_down, NI_up_og, NI_down_og, Cf_Rup_og,  Cf_Rdown_og, R_bids, Lambda_R, hour;
                             history = res_BM.history.history_insample, max_iter = 100, penalty_t = 1000 )
time4 = time()
println("Time for GDCA k-fold simulation: ", round(time4 - time3, digits=2), " seconds.")

Fold: 1
nb of variables 1323
nb of constraints 3253
Binary variables: 140
TA_bin_GDCA time: 72.58200001716614 seconds
nb of variables 1183
nb of constraints 3103
Binary variables: 0
TA_bin_LP_GDCA time: 18.938999891281128 seconds
Fold: 2
nb of variables 1323
nb of constraints 3253
Binary variables: 140
TA_bin_GDCA time: 30.76900005340576 seconds
nb of variables 1183
nb of constraints 3103
Binary variables: 0
TA_bin_LP_GDCA time: 0.12199997901916504 seconds
Fold: 3
nb of variables 1323
nb of constraints 3253
Binary variables: 140
TA_bin_GDCA time: 23.253999948501587 seconds
nb of variables 1183
nb of constraints 3103
Binary variables: 0
TA_bin_LP_GDCA time: 2.374000072479248 seconds
Fold: 4
nb of variables 1323
nb of constraints 3253
Binary variables: 140
TA_bin_GDCA time: 181.04299998283386 seconds
nb of variables 1183
nb of constraints 3103
Binary variables: 0
TA_bin_LP_GDCA time: 0.10599994659423828 seconds
Fold: 5
nb of variables 1323
nb of constraints 3253
Binary variables: 140
TA_

#### Saving the Results

In [10]:
# Assume your results dict is called `results`
data_BM = namedtuple_to_dict(res_BM)
data_bin = namedtuple_to_dict(res_GDCA)

@save "export/data_BM.jld2" data_BM
@save "export/data_bin.jld2" data_bin
@save "export/reserve_bids.jld2" R_bids

npzwrite("export/Constants.npz", Dict(
    "q_up" => q_up, 
    "q_down" => q_down,
    "NI_up" => NI_up_og,
    "NI_down" => NI_down_og,
    "Cp" => Cp,
    "Cf" => Cf,
    "Cr" => Cr,
    "Cf_Rup" => Cf_Rup_og,
    "Cf_Rdown" => Cf_Rdown_og,))

In [12]:
savefig(fig_down, "../Figures/Results/MOC/MOC_down_reserve_market.pdf")
savefig(fig_up, "../Figures/Results/MOC/MOC_up_reserve_market.pdf")

"c:\\Users\\Gloria\\Github\\Full-cost-balancing-\\Figures\\Results\\MOC\\MOC_up_reserve_market.pdf"